# Relationship between income and private health insurance

The goal is to investigate the quantitative effect of income on the procurement of private health insurance using US census data. The target of inference is the logistic regression coefficient when regressing the binary indicator of health insurance on income. The data from California in the year 2019 is downloaded through the Folktables interface (1). Predictions of health insurance are made by training a gradient boosting tree via XGBoost (2) on the previous year’s data.

1. F. Ding, M. Hardt, J. Miller, L. Schmidt, “Retiring adult: New datasets for fair machine learning” in Advances in Neural Information Processing Systems 34 (2021), pp. 6478–6490.
2. T. Chen, C. Guestrin, “XGBoost: A scalable tree boosting system” in Proceedings of the 22nd ACM SIGKDD International Conference on Knowledge Discovery and Data Mining (2016), pp. 785–794.

### Import necessary packages

In [1]:
%load_ext autoreload
%autoreload 2
import os, sys

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir)))
import numpy as np
import pandas as pd
from ppi_py.datasets import load_dataset
from ppi_py import eff_ppi_logistic_ci_tuned
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
from scipy.optimize import brentq
import matplotlib.pyplot as plt
import seaborn as sns

### Import the census healthcare data set

Load the data. The data set contains reported indicators of health insurance (```Y```), predicted indicators of health insurance (```Yhat```), and reported income (```X```).

In [2]:
dataset_folder = "./data/"
data = load_dataset(dataset_folder, "census_healthcare")
Y_total = data["Y"]
Yhat_total = data["Yhat"]
X_total = data["X"]

### Problem setup

Specify the error level (```alpha```), range of values for the labeled data set size (```ns```), and number of trials (```num_trials```).

Compute the ground-truth value of the estimand.

In [3]:
alpha = 0.05
coord = 0  # Choose between 0, 1
n_total = Y_total.shape[0]  # Total number of labeled examples
ns = np.array([500, 1000, 1500, 2000, 2500]).astype(
    int
)  # Test for different numbers of labeled ballots
num_trials = 100
optimizer_options = {
    'ftol': 1e-5,
    'gtol': 1e-5,
    'maxls': 10000,
    'maxiter': 10000
}
# Compute ground truth
theta_star = (
    LogisticRegression(
        penalty=None,
        solver="lbfgs",
        max_iter=10000,
        tol=1e-15,
        fit_intercept=False,
        )
        .fit(X_total, Y_total)
        .coef_.squeeze()
)
savename = 'census_healthcare_tuned'

### Construct intervals

Form confidence intervals for all methods and problem parameters. A dataframe with the following columns is formed:
1. ```method``` (one of ```PPI```, ```Classical```, and ```Imputation```)
2. ```n``` (labeled data set size, takes values in ```ns```)
3. ```lower``` (lower endpoint of the confidence interval)
4. ```upper``` (upper endpoint of the confidence interval)
5. ```trial``` (index of trial, goes from ```0``` to ```num_trials-1```)

In [ ]:
# Run prediction-powered inference and classical inference for many values of n
results = []
for j in tqdm(range(num_trials)):
    for i in range(ns.shape[0]):
        # Prediction-Powered Inference
        n = ns[i]
        rand_idx = np.random.permutation(n_total)
        _X, _X_unlabeled = X_total[rand_idx[:n]], X_total[rand_idx[n:]]
        _Y, _Y_unlabeled = Y_total[rand_idx[:n]], Y_total[rand_idx[n:]]
        _Yhat, _Yhat_unlabeled = (
            Yhat_total[rand_idx[:n]],
            Yhat_total[rand_idx[n:]],
        )
        ppi_ci_tuned = eff_ppi_logistic_ci_tuned(_X, _Y, _Yhat, _X_unlabeled, _Yhat_unlabeled, alpha=alpha, coord=coord, optimizer_options=optimizer_options)
        ppi_ci = eff_ppi_logistic_ci_tuned(_X, _Y, _Yhat, _X_unlabeled, _Yhat_unlabeled, alpha=alpha, lhat=1, optimizer_options=optimizer_options)
        classical_ci = eff_ppi_logistic_ci_tuned(_X, _Y, _Yhat, _X_unlabeled, _Yhat_unlabeled, alpha=alpha, lhat=0, optimizer_options=optimizer_options)

        # Append results
        results += [
            pd.DataFrame(
                [
                    {
                        "method": "PPI",
                        "n": n,
                        "lower": ppi_ci[0][coord],
                        "upper": ppi_ci[1][coord],
                        "included": (ppi_ci_tuned[0][coord] <= theta_star[coord]) & (ppi_ci_tuned[1][coord] >= theta_star[coord]),
                        "trial": j,
                    }
                ]
            )
        ]
        
        results += [
            pd.DataFrame(
                [
                    {
                        "method": "classical",
                        "n": n,
                        "lower": classical_ci[0][coord],
                        "upper": classical_ci[1][coord],
                        "included": (classical_ci[0][coord] <= theta_star[coord]) & (classical_ci[1][coord] >= theta_star[coord]),
                        "trial": j,
                    }
                ]
            )
        ]

        results += [
            pd.DataFrame(
                [
                    {
                        "method": "tuned PPI",
                        "n": n,
                        "lower": ppi_ci_tuned[0][coord],
                        "upper": ppi_ci_tuned[1][coord],
                        "included": (ppi_ci_tuned[0][coord] <= theta_star[coord]) & (ppi_ci_tuned[1][coord] >= theta_star[coord]),
                        "trial": j,
                    }
                ]
            )
        ]

df = pd.concat(results, axis=0, ignore_index=True)
df["width"] = df["upper"] - df["lower"]
os.makedirs('.cache/', exist_ok=True)
df.to_csv('.cache/' + savename + '.csv')

  6%|█████▌                                                                                       | 6/100 [08:20<2:10:26, 83.26s/it]

In [ ]:
palette = {'tuned PPI' : 'darkorange',
           'PPI' : '#83C980',
           'classical' : 'cornflowerblue'}
linewidth=2

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(8,2), sharex=True, sharey=False)
cvg_ax = axs[0]
sz_ax = axs[1]
sns.lineplot(ax=cvg_ax, data=df, x="n", y="included", linewidth=linewidth, errorbar=None, hue="method", legend=False, palette=palette, alpha=0.8)
sns.lineplot(ax=sz_ax, data=df, x="n", y="width", linewidth=linewidth, errorbar=None, hue="method", legend=True, palette=palette, alpha=0.8)
cvg_ax.set_ylabel("coverage")
cvg_ax.set_ylim([0.5,1.03])
cvg_ax.axhline(y=1-alpha, color="#888888", linestyle='dotted')
cvg_ax.set_xlabel("n")
sz_ax.set_ylabel("size")
sz_ax.set_xlabel("n")
sz_ax.legend_.set_title(None)
sns.despine(top=True, right=True)
plt.tight_layout()
os.makedirs('./plots',exist_ok=True)
plt.savefig('./plots/' + savename + '.pdf')